<a href="https://colab.research.google.com/github/sundaybest3/NounSmart_Streamlit/blob/main/countability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
%%capture
!pip install gradio
!pip install pandas
!pip install random

In [19]:
import gradio as gr
import pandas as pd
import random

# Load the CSV file
csv_url = "https://raw.githubusercontent.com/sundaybest3/NounSmart_Streamlit/main/nouns_CE_visang.csv"
df = pd.read_csv(csv_url)

# Inspect the columns
expected_columns = ["Unit", "Word", "Countability"]
missing_columns = [col for col in expected_columns if col not in df.columns]
if missing_columns:
    raise ValueError(f"The following required columns are missing in the CSV: {', '.join(missing_columns)}")

# Initialize game state
game_state = {
    "nickname": None,
    "unit": None,
    "current_noun": None,
    "remaining_nouns": None,
    "score": 0,
    "trials": 0,
}

# Function to filter nouns by unit and prepare the list of nouns
def initialize_nouns(unit):
    filtered_df = df[df["Unit"] == unit]
    if filtered_df.empty:
        return "No nouns available for this unit. Please select a different unit."
    game_state["remaining_nouns"] = filtered_df.sample(frac=1).to_dict(orient="records")
    return show_random_noun()

# Function to show a random noun
def show_random_noun():
    if not game_state["remaining_nouns"]:
        return f"🎉 Great job, {game_state['nickname']}! All nouns have been answered correctly. (Score: {game_state['score']}/{game_state['trials']})"

    game_state["current_noun"] = game_state["remaining_nouns"].pop()
    return f"Noun: {game_state['current_noun']['Word']}"

# Function to check user's answer
def check_answer(user_choice):
    if not game_state.get("current_noun"):
        return "Please click 'Show the Noun' first."

    correct_answer = game_state["current_noun"]["Countability"].strip().lower()
    game_state["trials"] += 1

    if user_choice.lower() == correct_answer:
        game_state["score"] += 1
        feedback = f"Correct! {game_state['current_noun']['Word']} is {correct_answer} in unit {game_state['unit']}."
    else:
        # Add the noun back to the remaining list for retry
        game_state["remaining_nouns"].insert(0, game_state["current_noun"])
        feedback = f"Incorrect. {game_state['current_noun']['Word']} is actually {correct_answer}. It will appear again."

    if not game_state["remaining_nouns"]:
        feedback += f" 🎉 Great job, {game_state['nickname']}! All nouns have been answered correctly. (Score: {game_state['score']}/{game_state['trials']})"

    return feedback + f" (Score: {game_state['score']} / {game_state['trials']})"

# Gradio interface
def main_interface(nickname, unit, user_choice=None):
    game_state["nickname"] = nickname
    game_state["unit"] = unit

    if game_state["remaining_nouns"] is None:
        return initialize_nouns(unit), ""

    if user_choice is None:
        return show_random_noun(), f"Score: {game_state['score']} / {game_state['trials']}"

    feedback = check_answer(user_choice)
    return feedback, feedback

# Gradio app components
with gr.Blocks() as app:
    nickname = gr.Textbox(label="Enter your nickname:")
    unit = gr.Dropdown(choices=df["Unit"].dropna().unique().tolist(), label="Select your unit:")
    show_button = gr.Button("Show the Noun")
    noun_display = gr.Textbox(label="Noun:", interactive=False)
    user_choice = gr.Radio(choices=["Countable", "Uncountable"], label="Is the noun countable or uncountable?")
    submit_button = gr.Button("Submit Answer")
    feedback_display = gr.Textbox(label="Feedback and Score:", interactive=False)

    show_button.click(fn=main_interface, inputs=[nickname, unit], outputs=[noun_display, feedback_display])
    submit_button.click(fn=main_interface, inputs=[nickname, unit, user_choice], outputs=[feedback_display, feedback_display])

app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://923ffa804428a8a7fa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [14]:
import gradio as gr
import pandas as pd
import random

# Load the CSV file
csv_url = "https://raw.githubusercontent.com/sundaybest3/NounSmart_Streamlit/main/nouns_CE_visang.csv"
df = pd.read_csv(csv_url)

# Inspect the columns
expected_columns = ["Unit", "Word", "Countability"]
missing_columns = [col for col in expected_columns if col not in df.columns]
if missing_columns:
    raise ValueError(f"The following required columns are missing in the CSV: {', '.join(missing_columns)}")

# Initialize game state
game_state = {
    "nickname": None,
    "unit": None,
    "current_noun": None,
    "score": 0,
    "trials": 0,
}

# Function to filter nouns by unit and show a random noun
def show_random_noun(unit):
    filtered_df = df[df["Unit"] == unit]
    if filtered_df.empty:
        return "No nouns available for this unit. Please select a different unit."

    noun_data = filtered_df.sample().iloc[0]
    game_state["current_noun"] = {
        "noun": noun_data["Word"],
        "countability": noun_data["Countability"].strip().lower(),
    }
    return f"Noun: {game_state['current_noun']['noun']}"

# Function to check user's answer
def check_answer(user_choice):
    if not game_state.get("current_noun"):
        return "Please click 'Show the Noun' first."

    correct_answer = game_state["current_noun"]["countability"]
    game_state["trials"] += 1

    if user_choice.lower() == correct_answer:
        game_state["score"] += 1
        feedback = f"✅Correct! {game_state['current_noun']['noun']} is {correct_answer} in unit {game_state['unit']}. Click 'Show the Noun' button to continue."
    else:
        feedback = f"❌Incorrect. {game_state['current_noun']['noun']} is actually {correct_answer}. The noun will appear again.Click 'Show the Noun' button to continue."

    return feedback

# Gradio interface
def main_interface(nickname, unit, user_choice=None):
    game_state["nickname"] = nickname
    game_state["unit"] = unit

    if user_choice is None:
        return show_random_noun(unit), f"Score: {game_state['score']} / {game_state['trials']}"

    feedback = check_answer(user_choice)
    return feedback, f"Score: {game_state['score']} / {game_state['trials']}"

# Gradio app components
with gr.Blocks() as app:
    nickname = gr.Textbox(label="Enter your nickname:")
    unit = gr.Dropdown(choices=df["Unit"].dropna().unique().tolist(), label="Select your unit:")
    show_button = gr.Button("Show the Noun")
    noun_display = gr.Textbox(label="Noun:", interactive=False)
    user_choice = gr.Radio(choices=["Countable", "Uncountable"], label="Is the noun countable or uncountable?")
    submit_button = gr.Button("Submit Answer")
    feedback_display = gr.Textbox(label="Feedback:", interactive=False)
    score_display = gr.Textbox(label="Score:", interactive=False)

    show_button.click(fn=main_interface, inputs=[nickname, unit], outputs=[noun_display, score_display])
    submit_button.click(fn=main_interface, inputs=[nickname, unit, user_choice], outputs=[feedback_display, score_display])

app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4cb2b54f539b8c7f16.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
